In [23]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import scipy
import os
from multiprocessing import Pool
from tqdm import tqdm
from sklearn.model_selection import cross_validate
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [24]:
datasetPath = '../../NumpyImages/'
tagFile = './imagePerTag.npy'

In [25]:
#Constants
classes = ['Moon','Day','ISS Structure','Volcano','Surface','Sun','Agriculture','Solar Panels','Inside ISS','Aurora','Clouds','Night','Atmosphere','Spacecraft','Deployed satellite','Movie','Hurricane','Solar Eclipse','Cupola','Dock Undock']

In [34]:
tags = np.load(tagFile).item()
tagList = tags.keys()
print(tagList)

dict_keys(['Canada', 'Cartagena', 'Therthar Lake', 'Moon', 'Collections', 'San Francisco', 'geo', 'Harvey', 'Day', 'Spain', 'Norfolk', 'Volga River', 'Lampedusa', 'Nile', 'Dardanelles', 'ISS Structure', 'Lagoon', 'Volcano', 'Marseille', 'Al Wahda Dam', 'Sinai Peninsula', 'Linosa', 'Washington', 'Surface', 'Agnano Crater', 'Alicudi Island', 'Agriculture', 'Turkey', 'Gallipoli', 'Sun', 'Solar Panels', 'Inside ISS', 'France', 'Castellammare del Golfo', '#WinEarthFavs', 'Arizona', 'Trapani', 'Aurora', 'San Diego', 'Park', 'Massachusetts', 'Los Angeles', 'BadBlankBlurry', 'Ukraine', 'Clouds', 'Florida', 'United States', 'Cape Cod', 'Russia', 'North America', 'Mt. Rainier', 'Chad', 'Cape Canaveral', 'Crimean Peninsula', 'Canary Islands', 'Hungary', 'Sunrise Sunset', 'Lake Balaton', 'Salton Sea', 'Aeolian Islands', 'Mt. Vesuvius', 'Asia', 'Gibraltar', 'Europe', 'Algeria', 'Sardinia', 'California', 'Delta', 'Iraq', 'El Ejido', 'Glen Canyon', 'Khenifiss National Park', 'Denmark', 'Morocco', 'Ni

In [27]:
def convertFilePath(imageList):
    images = []
    for img in imageList:
        imageFileName = img.split('/').pop()
        imageFileName += '.npy'
        images.append(imageFileName)
    return images

In [28]:
def loadImagesForTag(tag):
    positiveImages = tags[tag]
    negtiveImages = []
    for negitiveTag in tags:
        if(negitiveTag != tag):
            negtiveImages = sum([negtiveImages,tags[negitiveTag]],[])
    positiveImages = set(positiveImages)
    negtiveImages = set(negtiveImages)
    negtiveImages = negtiveImages - positiveImages
    return convertFilePath(list(positiveImages)), convertFilePath(list(negtiveImages))

In [29]:
def loadDataSet(positiveImages,negtiveImages):
    X = []
    y = []
    for positiveImage in positiveImages:
        X.append(np.load(datasetPath + positiveImage))
        y.append('true')
        
    for negtiveImage in negtiveImages:
        X.append(np.load(datasetPath + negtiveImage))
        y.append('false')
        
    return X,y

In [30]:
names = ["Nearest Neighbors", 
         #"Linear SVM", 
         #"RBF SVM", 
         #"Gaussian Process",
         "Decision Tree", 
         "Random Forest", 
         "Neural Net", 
         "AdaBoost",
         "Naive Bayes",
         "SVM",
         #"QDA"
        ]

classifiers = [
    KNeighborsClassifier(3),
    #SVC(kernel="linear", C=0.025, probability=True),
    #SVC(gamma=2, C=1, probability=True),
    #GaussianProcessClassifier(1.0 * RBF(1.0)), #This took forever and only returned 12%
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(),
    GaussianNB(),
    SVC(probability=True)
    #QuadraticDiscriminantAnalysis() # Did not converage
]

In [31]:
def classify(tag,X,y):
    lines = []
    for name, clf in zip(names, classifiers):
        scores = cross_validate(clf, X,y, cv=5)
        mean = scores['test_score'].mean()
        std = scores['test_score'].std() * 2
        lines.append(tag + ', ' + name + ', ' + str(mean) + ', ' + str(std))
    return lines

In [32]:
def testTag(tag):
    pos, neg = loadImagesForTag(tag)
    X,y = loadDataSet(pos,neg)
    return classify(tag,X,y)

In [33]:
p = Pool()
results = list(tqdm(p.imap(testTag, tagList), total=len(tagList)))


  0%|          | 0/113 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:597: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:597: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:597: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/Library/Frameworks/P

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:597: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:597: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:597: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3

ValueError: The number of classes has to be greater than one; got 1

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:597: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:597: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:597: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:597: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:597: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:597: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:597: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:597: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:597: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:597: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:597: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
